# Setup OMOP Vocabulary Tables

Developed to run in Databricks Community Edition Environment using RunTime: 11.3 LTS (Scala 2.12, Spark 3.3.0)

Downloads and extracts OMOP vocabulary data sets and creates delta tables for each set.

In [0]:
%sh
wget http://hls-eng-data-public.s3.amazonaws.com/omop/OMOP-VOCAB.tar.gz -P /databricks/driver/omop_vocab
cd /databricks/driver/omop_vocab/
tar -xf OMOP-VOCAB.tar.gz

--2023-01-27 16:41:49--  http://hls-eng-data-public.s3.amazonaws.com/omop/OMOP-VOCAB.tar.gz
Resolving hls-eng-data-public.s3.amazonaws.com (hls-eng-data-public.s3.amazonaws.com)... 54.231.235.145, 52.216.50.49, 52.216.136.75, ...
Connecting to hls-eng-data-public.s3.amazonaws.com (hls-eng-data-public.s3.amazonaws.com)|54.231.235.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 928389381 (885M) [application/x-tar]
Saving to: ‘/databricks/driver/omop_vocab/OMOP-VOCAB.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  491K 30m46s
    50K .......... .......... .......... .......... ..........  0%  821K 24m35s
   100K .......... .......... .......... .......... ..........  0% 24.3M 16m35s
   150K .......... .......... .......... .......... ..........  0% 3.41M 13m31s
   200K .......... .......... .......... .......... ..........  0% 1.07M 13m35s
   250K .......... .......... .......... .......... ..........  0%  164M 11m20s
   300K .....

In [0]:
%sql
create database if not exists omop_vocab

In [0]:
from pyspark.sql.functions import to_date

for f in dbutils.fs.ls("file:/databricks/driver/omop_vocab"):
  if f.name != 'OMOP-VOCAB.tar.gz': # Skip the file we just downloaded
    table_name = f.name.replace(".csv.gz","")
    df = spark.read.csv(f.path, inferSchema=True, header=True, dateFormat="yyyy-MM-dd")
    if table_name in ["CONCEPT","CONCEPT_RELATIONSHIP","DRUG_STRENGTH"]:
      df = df.withColumn('valid_start_date', to_date(df.valid_start_date,'yyyy-MM-dd')).withColumn('valid_end_date', to_date(df.valid_end_date,'yyyy-MM-dd'))
      
    df.write.format('delta').mode('overwrite').option('overwriteSchema','true').saveAsTable('omop_vocab.'+table_name)

#### Example Usage

In [0]:
%sql
select
  *
from
  omop_vocab.concept_synonym as a
  inner join omop_vocab.concept as b on a.concept_id = b.concept_id
limit
  50

concept_id,concept_synonym_name,language_concept_id,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
28,Observation type,4180186,28,Observation type,Metadata,Domain,Domain,null,OMOP generated,1970-01-01,2015-10-15,U
31,Relationship,4180186,31,Relationship,Metadata,Domain,Domain,null,OMOP generated,1970-01-01,2099-12-31,null
34,Currency,4180186,34,Currency,Metadata,Domain,Domain,null,OMOP generated,1970-01-01,2099-12-31,null
53,Condition/Drug,4180186,53,Condition/Drug,Metadata,Domain,Domain,null,OMOP generated,1970-01-01,2099-12-31,null
8478,avidity index,4180186,8478,avidity index,Unit,UCUM,Unit,S,{ai},1970-01-01,2099-12-31,null
8482,pH,4180186,8482,pH,Unit,UCUM,Unit,S,pH,1970-01-01,2099-12-31,null
8500,SAU,4180186,8500,SAU,Unit,UCUM,Unit,null,{sau},1970-01-01,2014-01-03,U
8501,SGU,4180186,8501,SGU,Unit,UCUM,Unit,null,{sgu},1970-01-01,2014-01-03,U
8514,% REF,4180186,8514,% REF,Unit,UCUM,Unit,null,%{REF},1970-01-01,2014-01-03,D
8534,Motile,4180186,8534,Motile,Unit,UCUM,Unit,null,{motile},1970-01-01,2014-01-03,U
